# Restricted Boltzman machine

In [4]:
import numpy as np

## Data

In [58]:
patterns = np.zeros((14,3,3))

patterns[1] = np.array([[1,0,0],[1,0,0],[1,0,0]])
patterns[2] = np.array([[0,1,0],[0,1,0],[0,1,0]])
patterns[3] = np.array([[0,0,1],[0,0,1],[0,0,1]])
patterns[4] = np.array([[1,1,0],[1,1,0],[1,1,0]])
patterns[5] = np.array([[0,1,1],[0,1,1],[0,1,1]])
patterns[6] = np.array([[1,0,1],[1,0,1],[1,0,1]])
patterns[7] = np.array([[1,1,1],[1,1,1],[1,1,1]])
patterns[8] = np.array([[1,1,1],[0,0,0],[0,0,0]])
patterns[9] = np.array([[0,0,0],[1,1,1],[0,0,0]])
patterns[10] = np.array([[0,0,0],[0,0,0],[1,1,1]])
patterns[11] = np.array([[1,1,1],[1,1,1],[0,0,0]])
patterns[12] = np.array([[0,0,0],[1,1,1],[1,1,1]])
patterns[13] = np.array([[1,1,1],[1,1,1],[0,0,0]])

In [61]:
patterns = patterns.reshape(-1,9)

In [33]:
class DataLoader:
    
    def __init__(self,data,po):
        self.data = data
        self.po = po
    
    def __iter__(self):
        return self

    def __next__(self):
        index = np.random.choice(range(len(self.data)), self.po, replace=False)
        return self.data[index]

In [103]:
class RBM:
    def __init__(self, nv, nh, beta=1, cd_k=1):   
        self.W = np.zeros((nh,nv))
        self.teta_v = np.zeros(nv)
        self.teta_h = np.zeros(nh)
        self.cd_k = cd_k 
        self.nv = nv
        self.nh = nh
        self.beta = beta
    
    def _p(self,b):
        return 1/(1+np.exp(-self.beta*b))         
    
    def activate_neuron_h(self,x):
        b_h = self.W@x - self.teta_h
        p_h= self._p(b_h)
        h = np.zeros(self.nh)
        for i in range(self.nh):
            h[i] = np.random.choice([-1,1],p=[p_h[i],1-p_h[i]])

        return h,b_h

    def activate_neuron_v(self,x):
        b_v = h.T@self.W - self.teta_v
        p_v = self._p(b_v)
        v = np.zeros(self.nv)
        for j in range(self.nh):
            v[j] = np.random.choice([-1,1],p=[p_v[i],1-p_v[i]])

        return v,b_v
    

In [104]:
N = 9
M = 4
rbm = RBM(N,M)
po = 5
data = DataLoader(patterns,po)

In [105]:
rbm.W.shape

(4, 9)

In [117]:
%%time
epochs = 10
lr = 0.1
for v in range(epochs):
    dW = np.zeros(rbm.W.shape)
    dTeta_h = np.zeros(rbm.teta_h.shape)
    dTeta_v = np.zeros(rbm.teta_v.shape)
    
    po_patterns = next(data)

    for x in po_patterns:
        h,b_0 = activate_neuron_h(rbm,x)
        for t in range(rbm.cd_k):
            v,_ = rbm.activate_neuron_v(x)
            h,b_k = rbm.activate_neuron_h(v)
        
    
    for m in range(rbm.nh):
        for n in range(rbm.nv):
            dW[m,n] = dW[m,n] + lr*(np.tanh(b_0[m])*x[n] - np.tanh(b_k[m]*v[n]))
            dTeta_v[n] = dTeta_v[n] + lr*(x[n]-v[n])
            dTeta_h[m] = dTeta_h[m] + lr*(np.tanh(b_0[m])-np.tanh(b_k[m])*v[n])
    
    rbm.W = rbm.W + dW
    rbm.teta_h = rbm.teta_h + dTeta_h
    rbm.teta_v = rbm.teta_v + dTeta_v


CPU times: user 27.5 ms, sys: 6.55 ms, total: 34.1 ms
Wall time: 30.9 ms
